<a href="https://colab.research.google.com/github/ShanmukhiKairuppala/Diabetic_Retinopathy_Detection/blob/main/HyperParameterTuning_%2B_CLAHE_on_DeepVision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp /content/drive/MyDrive/ColabNotebooks/Kaggle_API_Credentials/kaggle.json  ~/.kaggle/kaggle.json

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d mariaherrerot/aptos2019

Dataset URL: https://www.kaggle.com/datasets/mariaherrerot/aptos2019
License(s): unknown
100% 8.00G/8.01G [01:46<00:00, 109MB/s] 
100% 8.01G/8.01G [01:46<00:00, 81.0MB/s]


In [ ]:
!unzip aptos2019.zip -d Dataset/

Archive:  aptos2019.zip
  inflating: Dataset/test.csv        
  inflating: Dataset/test_images/test_images/e4dcca36ceb4.png  
  inflating: Dataset/test_images/test_images/e4e343eaae2a.png  
  inflating: Dataset/test_images/test_images/e4f12411fd85.png  
  inflating: Dataset/test_images/test_images/e50b0174690d.png  
  inflating: Dataset/test_images/test_images/e5197d77ec68.png  
  inflating: Dataset/test_images/test_images/e529c5757d64.png  
  inflating: Dataset/test_images/test_images/e52ed5c29c5e.png  
  inflating: Dataset/test_images/test_images/e540d2e35d15.png  
  inflating: Dataset/test_images/test_images/e55188915f9d.png  
  inflating: Dataset/test_images/test_images/e580676516b0.png  
  inflating: Dataset/test_images/test_images/e582e56e7942.png  
  inflating: Dataset/test_images/test_images/e594c19e2e1d.png  
  inflating: Dataset/test_images/test_images/e599151ca14b.png  
  inflating: Dataset/test_images/test_images/e59c5f345bb0.png  
  inflating: Dataset/test_images/test_imag

In [ ]:
import os
import cv2
import pandas as pd
import numpy as np
from skimage import io, img_as_ubyte
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt


In [ ]:
def preprocess_image(image, size=(256, 256)):
    if image.ndim == 2 or image.max() == image.min():
        print("Skipping image: Image is either grayscale or completely black.")
        return None

    if image.ndim == 3 and image.shape[2] == 3:
        lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
        l, a, b = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        cl = clahe.apply(l)
        limg = cv2.merge((cl, a, b))
        image = cv2.cvtColor(limg, cv2.COLOR_LAB2RGB)
    elif image.ndim == 2:
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        image = clahe.apply(image)

    resized_image = cv2.resize(image, size)
    return img_as_ubyte(resized_image)

def augment_image(image, base_filename, output_dir, retinopathy=True):
    transformations = [image]
    if retinopathy:
        for angle in [90, 120, 180, 270]:
            rotated_image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
            transformations.append(rotated_image)
    mirrored_image = cv2.flip(image, 1)
    transformations.append(mirrored_image)
    augmented_filenames = []
    for idx, transformed_image in enumerate(transformations):
        transformed_filename = f"{base_filename}_{idx}.png"
        transformed_path = os.path.join(output_dir, transformed_filename)
        io.imsave(transformed_path, transformed_image)
        augmented_filenames.append(transformed_filename)
    return augmented_filenames

def preprocess_and_augment(data_dir, output_dir, labels_file, augmented_csv):
    df = pd.read_csv(labels_file)
    augmented_data = []
    for level in df['diagnosis'].unique():
        class_dir = os.path.join(output_dir, str(level))
        if not os.path.exists(class_dir):
            os.makedirs(class_dir)
    for idx, row in df.iterrows():
        image_path = os.path.join(data_dir, row['id_code'] + '.png')
        image = io.imread(image_path)
        preprocessed_image = preprocess_image(image)
        if preprocessed_image is not None:
            retinopathy = row['diagnosis'] > 0
            class_dir = os.path.join(output_dir, str(row['diagnosis']))
            augmented_filenames = augment_image(preprocessed_image, row['id_code'], class_dir, retinopathy)
            for filename in augmented_filenames:
                augmented_data.append({'image': filename, 'level': row['diagnosis']})
    augmented_df = pd.DataFrame(augmented_data)
    augmented_df.to_csv(augmented_csv, index=False)


In [ ]:
# Example usage
preprocess_and_augment('/content/Dataset/train_images/train_images', '/content/Dataset/augmented/Training', '/content/Dataset/train_1.csv', '/content/Dataset/augmented_train_labels.csv')
preprocess_and_augment('/content/Dataset/test_images/test_images', '/content/Dataset/augmented/Testing', '/content/Dataset/test.csv', '/content/Dataset/augmented_test_labels.csv')
preprocess_and_augment('/content/Dataset/val_images/val_images', '/content/Dataset/augmented/Validation', '/content/Dataset/valid.csv', '/content/Dataset/augmented_validation_labels.csv')


In [ ]:
# Load pre-trained InceptionV3 model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(256, 256, 3))


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(5, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


87910968/87910968 [==============================] - 1s 0us/step


In [ ]:

train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/Dataset/augmented/Training',
    target_size=(256, 256),
    batch_size=32,
    class_mode='sparse')

validation_generator = validation_datagen.flow_from_directory(
    '/content/Dataset/augmented/Validation',
    target_size=(256, 256),
    batch_size=32,
    class_mode='sparse')


Found 11844 images belonging to 5 classes.
Found 1508 images belonging to 5 classes.


In [ ]:

model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=5)


Epoch 1/5
370/370 [==============================] - 2004s 5s/step - loss: 1.2144 - accuracy: 0.5876 - val_loss: 0.8813 - val_accuracy: 0.6576
Epoch 2/5
370/370 [==============================] - 1998s 5s/step - loss: 0.7833 - accuracy: 0.6848 - val_loss: 0.8953 - val_accuracy: 0.6449
Epoch 3/5
370/370 [==============================] - 1959s 5s/step - loss: 0.6813 - accuracy: 0.7320 - val_loss: 0.9147 - val_accuracy: 0.6489
Epoch 4/5
370/370 [==============================] - 1987s 5s/step - loss: 0.6021 - accuracy: 0.7642 - val_loss: 1.0410 - val_accuracy: 0.6144
Epoch 5/5
370/370 [==============================] - 2006s 5s/step - loss: 0.5180 - accuracy: 0.7977 - val_loss: 0.9633 - val_accuracy: 0.6689


In [ ]:

test_generator = test_datagen.flow_from_directory(
    '/content/Dataset/augmented/Testing',
    target_size=(256, 256),
    batch_size=32,
    class_mode='sparse',
    shuffle=False)

predictions = model.predict(test_generator)
y_pred = np.argmax(predictions, axis=1)
y_true = test_generator.classes

print("Accuracy: ", accuracy_score(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=['No DR', 'Mild DR', 'Moderate DR', 'Severe DR', 'Proliferative DR']))


Found 1400 images belonging to 5 classes.
44/44 [==============================] - 206s 5s/step
Accuracy:  0.6735714285714286
                  precision    recall  f1-score   support

           No DR       0.97      0.96      0.96       398
         Mild DR       0.48      0.33      0.39       180
     Moderate DR       0.59      0.85      0.70       522
       Severe DR       0.35      0.33      0.34       102
Proliferative DR       0.75      0.12      0.21       198

        accuracy                           0.67      1400
       macro avg       0.63      0.52      0.52      1400
    weighted avg       0.69      0.67      0.64      1400



In [ ]:

for layer in base_model.layers:
    layer.trainable = True

model.compile(optimizer=SGD(learning_rate=0.0001, momentum=0.9), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=4)


Epoch 1/4
370/370 [==============================] - 7254s 20s/step - loss: 0.6697 - accuracy: 0.7437 - val_loss: 0.8442 - val_accuracy: 0.6988
Epoch 2/4
370/370 [==============================] - 7240s 20s/step - loss: 0.4066 - accuracy: 0.8603 - val_loss: 0.8498 - val_accuracy: 0.7068
Epoch 3/4
370/370 [==============================] - 7211s 19s/step - loss: 0.2798 - accuracy: 0.9135 - val_loss: 0.8694 - val_accuracy: 0.7055
Epoch 4/4
  1/370 [..............................] - ETA: 1:52:19 - loss: 0.1327 - accuracy: 0.9688

KeyboardInterrupt: 

In [ ]:
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 127, 127, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 127, 127, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 127, 127, 32)         0         ['batch_normalization[0][0

In [ ]:

test_generator = test_datagen.flow_from_directory(
    '/content/Dataset/augmented/Testing',
    target_size=(256, 256),
    batch_size=32,
    class_mode='sparse',
    shuffle=False)

predictions = model.predict(test_generator)
y_pred = np.argmax(predictions, axis=1)
y_true = test_generator.classes

print("Accuracy: ", accuracy_score(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=['No DR', 'Mild DR', 'Moderate DR', 'Severe DR', 'Proliferative DR']))


Found 1400 images belonging to 5 classes.
44/44 [==============================] - 213s 5s/step
Accuracy:  0.7178571428571429
                  precision    recall  f1-score   support

           No DR       0.96      0.97      0.97       398
         Mild DR       0.53      0.52      0.53       180
     Moderate DR       0.67      0.82      0.73       522
       Severe DR       0.36      0.20      0.25       102
Proliferative DR       0.61      0.39      0.48       198

        accuracy                           0.72      1400
       macro avg       0.63      0.58      0.59      1400
    weighted avg       0.70      0.72      0.70      1400



In [ ]:
input_image_path = '/content/IDRiD_035.jpg'
preprocessed_image = preprocess_image(io.imread(input_image_path))  # Assuming preprocess_image is defined
image_array = np.expand_dims(preprocessed_image / 255.0, axis=0)
prediction = model.predict(image_array)
predicted_class = np.argmax(prediction)
print(f'Predicted DR Level: {predicted_class}')

1/1 [==============================] - 0s 179ms/step
Predicted DR Level: 4


In [ ]:
model.save('/content/drive/MyDrive/FineTunedModel/.')